In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv


In [2]:
lyric_database = pd.read_csv("/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv")
lyric_database.head(5)

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [3]:
db = lyric_database.sample(frac=0.01, random_state=1) #this database has 3 million entries
db = (
    db
    .loc[db["language"] == "en"]
    .loc[db["tag"] != "misc"]
    .loc[db["tag"] != ""]
    .loc[db["tag"] != None]
    .loc[db["lyrics"] != None]
    .loc[db["lyrics"] != ""]
    .loc[db["lyrics"] != "[Instrumental]"]
)
columns_of_interest = ['id', 'title','artist', 'lyrics', 'tag']
db = db[columns_of_interest]
db = db.sort_values(by="id")
db.head(10)

,id,title,artist,lyrics,tag
9,11,Lord You Know,Cam'ron,"[Chorus: Jaheim]\nNow Lord you know, just how ...",rap
219,216,Get It In Ohio,Cam'ron,"[Intro]\nWhat up, Midwest?\nThey forgot about ...",rap
240,243,Fuck tha Police,N.W.A,[Produced by Dr. Dre & DJ Yella]\n\n[Intro: Th...,rap
280,263,Kick in the Door,The Notorious B.I.G.,"[Segue from ""Hypnotize""]\nBiggie, Biggie, Bigg...",rap
370,365,Downtown Swinga 98,M.O.P.,[Intro]\nYeah\nCome on\nInternational\nDowntow...,rap
388,383,La Menage,Black Sheep,"[Intro: Dres]\nListen, hon, I mean, ah, as sla...",rap
740,710,Just Begun,Reflection Eternal,[Intro: Talib Kweli & The Singers Unlimited]\n...,rap
813,765,Aston Martin Music,Rick Ross,[Intro: Rick Ross]\nOh\nLet's pull the 2-seate...,rap
826,784,Stop the Music,P-Money (Producer),[Verse 1: Scribe]\nAnd I hope you feel my pain...,rap
919,871,Go Hard,DJ Khaled,"[Intro: Kanye West, DJ Khaled & T-Pain]\nAyy, ...",rap


In [4]:
print(db.tag.unique())
print(db.tag.value_counts(normalize=True))

['rap' 'rb' 'country' 'pop' 'rock']
tag
pop        0.430655
rap        0.298548
rock       0.196106
rb         0.046972
country    0.027719
Name: proportion, dtype: float64


In [5]:
import re
import nltk

def fix_lyrics(lyrics):
    parts = re.split(r"([\n\[\]\(\)])", lyrics)
    output = " ".join(filter(None, parts))
    output = re.sub(r"([?.,!:;])",'',output)
    output = re.sub(r"in'(?= \w|\.|,|$)","ing",output.lower())
    return output


db["lyrics"] = db["lyrics"].apply(fix_lyrics)
db = db.sort_values(by="id")

In [6]:
from nltk.tokenize import word_tokenize
db['tokens_raw'] = db['lyrics'].apply(lambda x: word_tokenize(x.lower()))
db.head(10)

,id,title,artist,lyrics,tag,tokens_raw
9,11,Lord You Know,Cam'ron,[ chorus jaheim ] \n now lord you know just ho...,rap,"[[, chorus, jaheim, ], now, lord, you, know, j..."
219,216,Get It In Ohio,Cam'ron,[ intro ] \n what up midwest \n they forgot ab...,rap,"[[, intro, ], what, up, midwest, they, forgot,..."
240,243,Fuck tha Police,N.W.A,[ produced by dr dre & dj yella ] \n \n [ intr...,rap,"[[, produced, by, dr, dre, &, dj, yella, ], [,..."
280,263,Kick in the Door,The Notorious B.I.G.,"[ segue from ""hypnotize"" ] \n biggie biggie bi...",rap,"[[, segue, from, ``, hypnotize, '', ], biggie,..."
370,365,Downtown Swinga 98,M.O.P.,[ intro ] \n yeah \n come on \n international ...,rap,"[[, intro, ], yeah, come, on, international, d..."
388,383,La Menage,Black Sheep,[ intro dres ] \n listen hon i mean ah as slam...,rap,"[[, intro, dres, ], listen, hon, i, mean, ah, ..."
740,710,Just Begun,Reflection Eternal,[ intro talib kweli & the singers unlimited ] ...,rap,"[[, intro, talib, kweli, &, the, singers, unli..."
813,765,Aston Martin Music,Rick Ross,[ intro rick ross ] \n oh \n let's pull the 2-...,rap,"[[, intro, rick, ross, ], oh, let, 's, pull, t..."
826,784,Stop the Music,P-Money (Producer),[ verse 1 scribe ] \n and i hope you feel my p...,rap,"[[, verse, 1, scribe, ], and, i, hope, you, fe..."
919,871,Go Hard,DJ Khaled,[ intro kanye west dj khaled & t-pain ] \n ayy...,rap,"[[, intro, kanye, west, dj, khaled, &, t-pain,..."


In [7]:
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
db['tokens_raw'] = db['tokens_raw'].apply(lambda x: [w for w in x if w not in stopwords])
db['lyrics_clean'] = db['tokens_raw'].apply(lambda x: ' '.join(x))

db.head(10)

,id,title,artist,lyrics,tag,tokens_raw,lyrics_clean
9,11,Lord You Know,Cam'ron,[ chorus jaheim ] \n now lord you know just ho...,rap,"[[, chorus, jaheim, ], lord, know, hard, try, ...",[ chorus jaheim ] lord know hard try live life...
219,216,Get It In Ohio,Cam'ron,[ intro ] \n what up midwest \n they forgot ab...,rap,"[[, intro, ], midwest, forgot, fourth, coast, ...",[ intro ] midwest forgot fourth coast ai n't n...
240,243,Fuck tha Police,N.W.A,[ produced by dr dre & dj yella ] \n \n [ intr...,rap,"[[, produced, dr, dre, &, dj, yella, ], [, int...",[ produced dr dre & dj yella ] [ intro doc dr ...
280,263,Kick in the Door,The Notorious B.I.G.,"[ segue from ""hypnotize"" ] \n biggie biggie bi...",rap,"[[, segue, ``, hypnotize, '', ], biggie, biggi...",[ segue `` hypnotize '' ] biggie biggie biggie...
370,365,Downtown Swinga 98,M.O.P.,[ intro ] \n yeah \n come on \n international ...,rap,"[[, intro, ], yeah, come, international, downt...",[ intro ] yeah come international downtown swi...
388,383,La Menage,Black Sheep,[ intro dres ] \n listen hon i mean ah as slam...,rap,"[[, intro, dres, ], listen, hon, mean, ah, sla...",[ intro dres ] listen hon mean ah slamming goo...
740,710,Just Begun,Reflection Eternal,[ intro talib kweli & the singers unlimited ] ...,rap,"[[, intro, talib, kweli, &, singers, unlimited...",[ intro talib kweli & singers unlimited ] 've ...
813,765,Aston Martin Music,Rick Ross,[ intro rick ross ] \n oh \n let's pull the 2-...,rap,"[[, intro, rick, ross, ], oh, let, 's, pull, 2...",[ intro rick ross ] oh let 's pull 2-seater ba...
826,784,Stop the Music,P-Money (Producer),[ verse 1 scribe ] \n and i hope you feel my p...,rap,"[[, verse, 1, scribe, ], hope, feel, pain, hop...",[ verse 1 scribe ] hope feel pain hope n't for...
919,871,Go Hard,DJ Khaled,[ intro kanye west dj khaled & t-pain ] \n ayy...,rap,"[[, intro, kanye, west, dj, khaled, &, t-pain,...",[ intro kanye west dj khaled & t-pain ] ayy go...


In [8]:
from sklearn.model_selection import train_test_split

X = db['lyrics_clean'].copy()
y = db['tag'].copy()

X_train_raw, X_val_raw, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=60)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
#import seaborn as sns

tfidf_vectorizer = TfidfVectorizer()
X_train = tfidf_vectorizer.fit_transform(X_train_raw).toarray()

# an alternative is to use term frequency:
#from sklearn.feature_extraction.text import CountVectorizer
#one_hot_vectorizer = CountVectorizer(binary=True)
#X_train = one_hot_vectorizer.fit_transform(X_train_raw))

print(f"X_train.shape = {X_train.shape}")
type(X_train)

X_train.shape = (24270, 79070)


numpy.ndarray

In [10]:
X_val = tfidf_vectorizer.transform(X_val_raw).toarray()
print(f"X_val.shape = {X_val.shape}")
type(X_val)

X_val.shape = (8090, 79070)


numpy.ndarray

In [11]:
print(f"pop songs = {len(db.loc[db['tag'] == 'pop'])}")
print(f"rap songs = {len(db.loc[db['tag'] == 'rap'])}")
print(f"rock songs = {len(db.loc[db['tag'] == 'rock'])}")
print(f"rb songs = {len(db.loc[db['tag'] == 'rb'])}")
print(f"country songs = {len(db.loc[db['tag'] == 'country'])}")

pop songs = 13936
rap songs = 9661
rock songs = 6346
rb songs = 1520
country songs = 897


In [12]:
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score


#models = []

#model = KNeighborsClassifier(n_neighbors=11)
#model = model.fit(X_train, y_train)

#predictions_train = model.predict(X_train)

#disp = ConfusionMatrixDisplay(confusion_matrix(y_train, predictions_train), display_labels='pop', 'rap', 'rock', 'rb', 'country')
#disp.plot()
#print(f"accuracy (on y_train): {accuracy_score(y_train, predictions_train):.4f}")
#models.append(model)

In [13]:
#from sklearn.metrics import log_loss

# A function to create and fit a RF with a specific number of trees
#def tuneModel(hyperparam_value):
#    rf_model = KNeighborsClassifier(n_neighbors=hyperparam_value) 
#    #rf_model = RandomForestClassifier(min_samples_split=hyperparam_value, random_state=1)
#    rf_model.fit(X_train, y_train)
#    y_train_pred_prob = rf_model.predict_proba(X_train)
#    y_train_pred = rf_model.predict(X_train)
#    y_val_pred_prob = rf_model.predict_proba(X_val)
#    y_val_pred = rf_model.predict(X_val)
#    train_loss = log_loss(y_train, y_train_pred_prob, labels=['pop', 'rap', 'rock', 'rb', 'country'])
#    train_acc = accuracy_score(y_train, y_train_pred)
#    val_loss = log_loss(y_val, y_val_pred_prob, labels=['pop', 'rap', 'rock', 'rb', 'country'])
#    val_acc = accuracy_score(y_val, y_val_pred)
#    return (train_loss, val_loss, train_acc, val_acc)

# Possible values for the hyperparameter are in the range of 5 to 150 (by 50)
#hyp_param_vals = range(7,13) # good values for n_estimators
#hyp_param_vals = [2,3] + list(range(5, 50, 10)) # good values for min_samples_split
#metrics = []

#for hp in hyp_param_vals:
#    metrics.append(tuneModel(hp))